In [2]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

#### Scrap the district table

In [3]:
import requests
website_url = requests.get('https://keylocation.sg/singapore/districts-map').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [4]:
tables = soup.findAll("table", { "class" : "table table-striped" })
# show tables
for table in tables:
    print ("###############")
    print (table.text[:100]) 

###############
DistrictAreasLocation1Marina AreaBoat Quay, Chinatown, Havelock Road, Marina Square, Raffles Place, 
###############
DistrictAreasLocation16Upper East CoastBayshore, Bedok, Chai Chee17Far EastChangi, Loyang, Pasir Ris
###############
DistrictAreasLocation19North EastHougang, Punggol, Sengkang20Ang Mo KioAng Mo Kio, Bishan, Braddell 
###############
DistrictAreasLocation22Far WestBoon Lay, Jurong, Tuas23North WestBukit Batok, Choa Chu Kang, Hillvie
###############
DistrictAreasLocation25Far NorthAdmiralty, Woodlands26NorthTagore, Yio Chu Kang27Far NorthAdmiralty,


In [5]:
import pandas as pd

In [6]:

for tn in range(len(tables)):
    table=tables[tn]

    # preinit list of lists
    rows=table.findAll("tr")
    row_lengths=[len(r.findAll(['th','td'])) for r in rows]
    ncols=max(row_lengths)
    nrows=len(rows)
    data=[]
    for i in range(nrows):
        rowD=[]
        for j in range(ncols):
            rowD.append('')
        data.append(rowD)
    
    # process html
    for i in range(len(rows)):
        row=rows[i]
        rowD=[]
        cells = row.findAll(["td","th"])
        for j in range(len(cells)):
            cell=cells[j]
            
            #lots of cells span cols and rows so lets deal with that
            cspan=int(cell.get('colspan',1))
            rspan=int(cell.get('rowspan',1))
            for k in range(rspan):
                for l in range(cspan):
                    data[i+k][j+l]+=cell.text
    
        data.append(rowD)
    data
    import codecs
    fname='/resources/data/output_t{}.csv'.format(tn)
    f = codecs.open(fname, 'w')#,encoding='utf-8')
    #f = codecs.open('/resources/data/Districts.csv', 'w')#,encoding='utf-8')
    for i in range(nrows):
        rowStr=','.join(data[i])
        rowStr=rowStr.replace('\n','')
        print (rowStr)
        rowStr=rowStr #.encode('unicode_escape')
        f.write(rowStr+'\n')             
        
    f.close()



District,Areas,Location
1,Marina Area,Boat Quay, Chinatown, Havelock Road, Marina Square, Raffles Place, Suntec City
2,CBD,Anson Road, Chinatown, Neil Road, Raffles Place, Shenton Way, Tanjong Pagar
3,Central South,Alexandra Road, Tiong Bahru, Queenstown
4,Keppel,Keppel, Mount Faber, Sentosa, Telok Blangah
5,South West,Buona Vista, Dover, Pasir Panjang, West Coast
6,City Hall,City Hall, High Street, North Bridge Road
7,Beach Road,Beach Road, Bencoolen Road, Bugis, Rochor
8,Little India,Little India, Farrer Park, Serangoon Road
9,Orchard,Cairnhill, Killiney, Leonie Hill, Orchard, Oxley
10,Tanglin,Balmoral, Bukit Timah, Grange Road, Holland, Orchard Boulevard, River Valley, Tanglin Road
11,Newton,Chancery, Bukit Timah, Dunearn Road, Newton
12,Toa Payoh,Balestier, Moulmein, Novena, Toa Payoh
13,Central East,Potong Pasir, Macpherson
14,Eunos,Eunos, Geylang, Kembangan, Paya Lebar
15,East Coast,Katong, Marine Parade, Siglap, Tanjong Rhu
21,Central West,Clementi, Upper Bukit Timah, Hume Avenu

#### Convert into a data frame

In [7]:
import pandas as pd
# define the dataframe columns
my_cols = ['District', 'Neighborhood', 'Loc1', 'Loc2','Loc3','Loc4','Loc5','Loc6','Loc7'] 
# instantiate the dataframe
#df_districts = pd.DataFrame(columns=my_cols)
csv_path='/resources/data/output_t0.csv'
df_t0 = pd.read_csv(csv_path, skiprows=[0], header=None, names=my_cols, engine='python')
df_t0
csv_path='/resources/data/output_t1.csv'
df_t1 = pd.read_csv(csv_path, skiprows=[0], header=None, names=my_cols, engine='python')
df_t1
csv_path='/resources/data/output_t2.csv'
df_t2 = pd.read_csv(csv_path, skiprows=[0], header=None, names=my_cols, engine='python')
df_t2
csv_path='/resources/data/output_t3.csv'
df_t3 = pd.read_csv(csv_path, skiprows=[0], header=None, names=my_cols, engine='python')
df_t3
csv_path='/resources/data/output_t4.csv'
df_t4 = pd.read_csv(csv_path, skiprows=[0], header=None, names=my_cols, engine='python')
df_t4
d=[df_t0, df_t1, df_t2, df_t3, df_t4]
df_D = pd.concat(d).reset_index(drop=True)
df_D
#csv_path='/resources/data/output_t0.csv'
#df_districts = pd.read_csv(csv_path)
#df_districts

,District,Neighborhood,Loc1,Loc2,Loc3,Loc4,Loc5,Loc6,Loc7
0,1,Marina Area,Boat Quay,Chinatown,Havelock Road,Marina Square,Raffles Place,Suntec City,None
1,2,CBD,Anson Road,Chinatown,Neil Road,Raffles Place,Shenton Way,Tanjong Pagar,None
2,3,Central South,Alexandra Road,Tiong Bahru,Queenstown,None,None,None,None
3,4,Keppel,Keppel,Mount Faber,Sentosa,Telok Blangah,None,None,None
4,5,South West,Buona Vista,Dover,Pasir Panjang,West Coast,None,None,None
5,6,City Hall,City Hall,High Street,North Bridge Road,None,None,None,None
6,7,Beach Road,Beach Road,Bencoolen Road,Bugis,Rochor,None,None,None
7,8,Little India,Little India,Farrer Park,Serangoon Road,None,None,None,None
8,9,Orchard,Cairnhill,Killiney,Leonie Hill,Orchard,Oxley,None,None
9,10,Tanglin,Balmoral,Bukit Timah,Grange Road,Holland,Orchard Boulevard,River Valley,Tanglin Road


#### Generate Latitude and Longitude of the district and add it to the existing data frame

In [8]:
csv_path='/resources/data/Coord.csv'
cols=['Latitude','Longitude']
df_coord = pd.read_csv(csv_path, skiprows=[0], header=None, names=cols, engine='python')
df_coord
df_district = pd.concat([df_D, df_coord], axis=1, join='inner')
df_district

,District,Neighborhood,Loc1,Loc2,Loc3,Loc4,Loc5,Loc6,Loc7,Latitude,Longitude
0,1,Marina Area,Boat Quay,Chinatown,Havelock Road,Marina Square,Raffles Place,Suntec City,None,1.280386,103.840644
1,2,CBD,Anson Road,Chinatown,Neil Road,Raffles Place,Shenton Way,Tanjong Pagar,None,1.278898,103.841915
2,3,Central South,Alexandra Road,Tiong Bahru,Queenstown,None,None,None,None,1.290475,103.852036
3,4,Keppel,Keppel,Mount Faber,Sentosa,Telok Blangah,None,None,None,1.281756,103.841699
4,5,South West,Buona Vista,Dover,Pasir Panjang,West Coast,None,None,None,1.283519,103.842310
5,6,City Hall,City Hall,High Street,North Bridge Road,None,None,None,None,1.285370,103.843173
6,7,Beach Road,Beach Road,Bencoolen Road,Bugis,Rochor,None,None,None,1.286180,103.843961
7,8,Little India,Little India,Farrer Park,Serangoon Road,None,None,None,None,1.285567,103.844849
8,9,Orchard,Cairnhill,Killiney,Leonie Hill,Orchard,Oxley,None,None,1.287341,103.844694
9,10,Tanglin,Balmoral,Bukit Timah,Grange Road,Holland,Orchard Boulevard,River Valley,Tanglin Road,1.288645,103.842163


In [9]:
D01_data = df_district[df_district['Neighborhood'] == 'Marina Area'].reset_index(drop=True)
D01_data.head()

,District,Neighborhood,Loc1,Loc2,Loc3,Loc4,Loc5,Loc6,Loc7,Latitude,Longitude
0,1,Marina Area,Boat Quay,Chinatown,Havelock Road,Marina Square,Raffles Place,Suntec City,None,1.280386,103.840644


#### Create a subset for the data frame to separate Districts 1, 2, 4 and 9

In [93]:
#df_for_rent=(df_district.loc[df_district['District'].isin(['1','2','4','6','9','10','11','12','13'])])
#df_for_rent
df_for_rent=(df_district.loc[df_district['District'].isin(['1','2','4','9'])])
df_for_rent
del df_for_rent['Loc7']
df_for_rent.reset_index(drop=True)

,District,Neighborhood,Loc1,Loc2,Loc3,Loc4,Loc5,Loc6,Latitude,Longitude
0,1,Marina Area,Boat Quay,Chinatown,Havelock Road,Marina Square,Raffles Place,Suntec City,1.280386,103.840644
1,2,CBD,Anson Road,Chinatown,Neil Road,Raffles Place,Shenton Way,Tanjong Pagar,1.278898,103.841915
2,4,Keppel,Keppel,Mount Faber,Sentosa,Telok Blangah,None,None,1.281756,103.841699
3,9,Orchard,Cairnhill,Killiney,Leonie Hill,Orchard,Oxley,None,1.287341,103.844694


#### For the all four districts generate a list of condos with name, nearest MRT, No. of Listing, Price Range

In [123]:
csv_path1='/resources/data/CL-D01.csv'
csv_path2='/resources/data/CL-D02.csv'
csv_path3='/resources/data/CL-D04.csv'
csv_path4='/resources/data/CL-D09.csv'
new_cols=['District','Neighborhood','Condo_Name','Nearest MRT','No_of_Listings','Price Range (SGD)']
CL_D01 = pd.read_csv(csv_path1, skiprows=[0], header=None, names=new_cols, engine='python')
CL_D02 = pd.read_csv(csv_path2, skiprows=[0], header=None, names=new_cols, engine='python')
CL_D04 = pd.read_csv(csv_path3, skiprows=[0], header=None, names=new_cols, engine='python')
CL_D09 = pd.read_csv(csv_path4, skiprows=[0], header=None, names=new_cols, engine='python')
all=[CL_D01, CL_D02, CL_D04, CL_D09]
CL_All = pd.concat(all).reset_index(drop=True)
#CL_All.drop()
CL_All.drop(CL_All.index[96:107], inplace=True)
CL_All

,District,Neighborhood,Condo_Name,Nearest MRT,No_of_Listings,Price Range (SGD)
30,4,Keppel,TERESA VILLE,CE: Harbourfront,3,1100
31,9,Orchard,NEWTON EDGE,DT: Newton,8,2300-3000
32,9,Orchard,ORCHARD SCOTTS,DT: Newton,2,4300-4500
33,9,Orchard,THE SCOTTS TOWER,DT: Newton,100,3180-6300
34,9,Orchard,VIDA,DT: Newton,121,2900-5100
35,9,Orchard,THE PEAK,DT: Newton,18,2700-3600
36,9,Orchard,THE VERMONT ON CAIRNHILL,DT: Newton,2,3000
37,9,Orchard,CAVENAGH HOSE,DT: Newton,1,2400
38,9,Orchard,DELFI ORCHARD,DT: Newton,2,3100-3200
39,9,Orchard,ORCHARD TOWERS,DT: Newton,3,1600-1800


#### Sort the data frame in descending order of listing

In [12]:
CL_All = CL_All.sort_values('No_of_Listings', ascending=False).reset_index(drop=True)
CL_All

,District,Neighborhood,Condo_Name,Nearest MRT,No_of_Listings,Price Range (SGD)
0,1,Marina Area,THE CLIFT,DT: Telok Ayer,283,2900-5500
1,1,Marina Area,MARINA ONE RESIDENCES,CE: Marina Bay,198,3000-4900
2,1,Marina Area,THE SAIL AT MARINA BAY,DT: Downtown,186,2200-5600
3,2,CBD,ICON,EW: Tanjong Pagar,167,2800-4800
4,1,Marina Area,V ON SHENTON,DT: Downtown,132,2800-5000
5,9,Orchard,VIDA,DT: Newton,121,2900-5100
6,9,Orchard,THE SCOTTS TOWER,DT: Newton,100,3180-6300
7,9,Orchard,RIVERSIDE 48,DT: Fort Canning,92,2800-3400
8,9,Orchard,SCOTTS SQUARE,NS: Orchard,81,3600-5500
9,9,Orchard,ESPADA,NS: Somerset,81,2500-4000


#### Filter top Condos where number of listings are more than 50

In [13]:
CL_Top = CL_All[CL_All['No_of_Listings'] > 50]
CL_Top
#temp2 = df[df["bin"] == 3]
#temp2 = temp2[~temp2["Def"] & temp2["days since"] > 7]

,District,Neighborhood,Condo_Name,Nearest MRT,No_of_Listings,Price Range (SGD)
0,1,Marina Area,THE CLIFT,DT: Telok Ayer,283,2900-5500
1,1,Marina Area,MARINA ONE RESIDENCES,CE: Marina Bay,198,3000-4900
2,1,Marina Area,THE SAIL AT MARINA BAY,DT: Downtown,186,2200-5600
3,2,CBD,ICON,EW: Tanjong Pagar,167,2800-4800
4,1,Marina Area,V ON SHENTON,DT: Downtown,132,2800-5000
5,9,Orchard,VIDA,DT: Newton,121,2900-5100
6,9,Orchard,THE SCOTTS TOWER,DT: Newton,100,3180-6300
7,9,Orchard,RIVERSIDE 48,DT: Fort Canning,92,2800-3400
8,9,Orchard,SCOTTS SQUARE,NS: Orchard,81,3600-5500
9,9,Orchard,ESPADA,NS: Somerset,81,2500-4000


We notice that in top list district 4 (Keppel area is ommitted)
#### Stored the name of top three districts in separate variable

In [80]:
df_Neighborhood=pd.DataFrame
df_Neighborhood=CL_Top['Neighborhood'].unique()
df_Neighborhood
#df_Neighborhood['Latitude']=df_for_rent[df_for_rent['Latitude'].index(0:4)]


array(['Marina Area', 'CBD', 'Orchard'], dtype=object)

#### Store your Foursquare Credentials

In [15]:
CLIENT_ID = 'UEKWPIJKYFQFHR2Z2DACXHDCBIL2N2UMOK4H4ZV44EWOJX5H' # your Foursquare ID
CLIENT_SECRET = 'PWCWYOOLUCMD4SNNXG3I2REZM3S3GU124UYUDG212ABAV1SE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UEKWPIJKYFQFHR2Z2DACXHDCBIL2N2UMOK4H4ZV44EWOJX5H
CLIENT_SECRET:PWCWYOOLUCMD4SNNXG3I2REZM3S3GU124UYUDG212ABAV1SE


#### Explore the first Neighborhood

In [16]:
df_for_rent_lat = df_for_rent.loc[0, 'Latitude'] # District latitude value
df_for_rent_lon = df_for_rent.loc[0, 'Longitude'] # District longitude value

df_for_rent_name = df_for_rent.loc[0, 'Neighborhood'] # District name

print('Latitude and longitude values of {} are {}, {}.'.format(df_for_rent_name, 
                                                               df_for_rent_lat, 
                                                               df_for_rent_lon))

Latitude and longitude values of Marina Area are 1.280386, 103.8406437.


#### Now, let's get the top 100 venues that are in Marina Area within a radius of 500 meters.

In [40]:
LIMIT = 100 # limit of number of venues returned by Foursquare API



radius = 500 # define radius



 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df_for_rent_lat, 
    df_for_rent_lon, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=UEKWPIJKYFQFHR2Z2DACXHDCBIL2N2UMOK4H4ZV44EWOJX5H&client_secret=PWCWYOOLUCMD4SNNXG3I2REZM3S3GU124UYUDG212ABAV1SE&v=20180605&ll=1.280386,103.8406437&radius=500&limit=100'

In [42]:
results = requests.get(url).json()


In [43]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()    

,name,categories,lat,lng
0,APIARY,Ice Cream Shop,1.279499,103.842294
1,Man Man 鰻満 Japanese Unagi Restaurant (Man Man ...,Japanese Restaurant,1.278876,103.841514
2,D.Bespoke,Speakeasy,1.278680,103.840897
3,Burnt Ends,Australian Restaurant,1.280502,103.841745
4,Kok Sen Restaurant,Chinese Restaurant,1.280315,103.841617


In [44]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

92 venues were returned by Foursquare.


### Create a function to get the list of nearby venues.

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [50]:
# type your answer here
main_venues = getNearbyVenues(names=df_Neighborhood,
                                   latitudes=df_for_rent['Latitude'],
                                   longitudes=df_for_rent['Longitude']
                                  )
main_venues.head()
main_venues

Marina Area
CBD
Orchard


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marina Area,1.280386,103.840644,APIARY,1.279499,103.842294,Ice Cream Shop
1,Marina Area,1.280386,103.840644,Man Man 鰻満 Japanese Unagi Restaurant (Man Man ...,1.278876,103.841514,Japanese Restaurant
2,Marina Area,1.280386,103.840644,D.Bespoke,1.278680,103.840897,Speakeasy
3,Marina Area,1.280386,103.840644,Burnt Ends,1.280502,103.841745,Australian Restaurant
4,Marina Area,1.280386,103.840644,Kok Sen Restaurant,1.280315,103.841617,Chinese Restaurant
5,Marina Area,1.280386,103.840644,Mad About Sucre,1.279782,103.840658,Dessert Shop
6,Marina Area,1.280386,103.840644,Esquina Tapas Bar,1.279976,103.842053,Tapas Restaurant
7,Marina Area,1.280386,103.840644,Potato Head Folk,1.280487,103.841699,Burger Joint
8,Marina Area,1.280386,103.840644,Neon Pigeon,1.278912,103.841543,Japanese Restaurant
9,Marina Area,1.280386,103.840644,kiyone+LIM,1.280821,103.840930,Nail Salon


In [47]:
print(main_venues.shape)

(292, 7)


In [48]:
main_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
CBD,100,100,100,100,100,100
Marina Area,92,92,92,92,92,92
Orchard,100,100,100,100,100,100


In [49]:
print('There are {} uniques categories.'.format(len(main_venues['Venue Category'].unique())))

There are 59 uniques categories.


## **Analyze Each Neighbourhood**

In [51]:
# one hot encoding
singapore_onehot = pd.get_dummies(main_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
singapore_onehot['Neighborhood'] = main_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [singapore_onehot.columns[-1]] + list(singapore_onehot.columns[:-1])
singapore_onehot = singapore_onehot[fixed_columns]

singapore_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Australian Restaurant,Bakery,Bar,Beer Bar,Beer Garden,Bookstore,Buddhist Temple,...,Speakeasy,Sporting Goods Shop,Sports Club,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,Marina Area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marina Area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Marina Area,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Marina Area,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marina Area,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
singapore_onehot.shape

(292, 60)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [53]:
singapore_grouped = singapore_onehot.groupby('Neighborhood').mean().reset_index()
singapore_grouped

,Neighborhood,American Restaurant,Asian Restaurant,Australian Restaurant,Bakery,Bar,Beer Bar,Beer Garden,Bookstore,Buddhist Temple,...,Speakeasy,Sporting Goods Shop,Sports Club,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,CBD,0.020000,0.030000,0.020000,0.030000,0.00,0.020000,0.020000,0.020000,0.01000,...,0.01000,0.01000,0.01000,0.01,0.020000,0.020000,0.01,0.000000,0.01000,0.01000
1,Marina Area,0.021739,0.032609,0.021739,0.021739,0.00,0.021739,0.021739,0.021739,0.01087,...,0.01087,0.01087,0.01087,0.00,0.021739,0.021739,0.00,0.021739,0.01087,0.01087
2,Orchard,0.010000,0.020000,0.020000,0.020000,0.01,0.020000,0.020000,0.020000,0.01000,...,0.01000,0.01000,0.01000,0.00,0.020000,0.020000,0.00,0.020000,0.02000,0.01000


In [54]:
singapore_grouped.shape

(3, 60)

In [118]:
num_top_venues = 21

for hood in singapore_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = singapore_grouped[singapore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CBD----
                   venue  freq
0                   Café  0.07
1      Korean Restaurant  0.06
2     Italian Restaurant  0.05
3                  Hotel  0.05
4     Chinese Restaurant  0.04
5             Food Court  0.04
6    Japanese Restaurant  0.04
7                 Bakery  0.03
8           Cocktail Bar  0.03
9            Coffee Shop  0.03
10      Asian Restaurant  0.03
11     French Restaurant  0.03
12      Ramen Restaurant  0.02
13            Restaurant  0.02
14    Spanish Restaurant  0.02
15      Tapas Restaurant  0.02
16              Tea Room  0.02
17  Gym / Fitness Center  0.02
18   American Restaurant  0.02
19          Dessert Shop  0.02
20              Beer Bar  0.02


----Marina Area----
                            venue  freq
0               Korean Restaurant  0.07
1                            Café  0.07
2              Italian Restaurant  0.05
3              Chinese Restaurant  0.05
4                      Food Court  0.04
5             Japanese Restaurant  0.04
6   

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [56]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 20 venues for each neighborhood.

In [59]:
import numpy as np
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = singapore_grouped['Neighborhood']

for ind in np.arange(singapore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,CBD,Café,Korean Restaurant,Hotel,Italian Restaurant,Chinese Restaurant,Food Court,Japanese Restaurant,Bakery,Cocktail Bar,...,French Restaurant,Asian Restaurant,Beer Bar,Beer Garden,Bookstore,Burger Joint,Australian Restaurant,Dessert Shop,Gym / Fitness Center,American Restaurant
1,Marina Area,Korean Restaurant,Café,Italian Restaurant,Chinese Restaurant,Food Court,Coffee Shop,Japanese Restaurant,Cocktail Bar,Dessert Shop,...,Bookstore,Beer Garden,Beer Bar,Bakery,Australian Restaurant,French Restaurant,Burger Joint,American Restaurant,Vegetarian / Vegan Restaurant,Tea Room
2,Orchard,Chinese Restaurant,Korean Restaurant,Hostel,Cocktail Bar,Café,Spa,Japanese Restaurant,Food Court,Seafood Restaurant,...,Spanish Restaurant,Coffee Shop,Hotel,Wine Bar,Vegetarian / Vegan Restaurant,Asian Restaurant,French Restaurant,Tea Room,Beer Bar,Dessert Shop


## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [103]:
# set number of clusters
kclusters = 3

singapore_grouped_clustering = singapore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(singapore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([0, 2, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 20 venues for each neighborhood.

In [104]:
# add latitude and longitude data
df_a=df_for_rent.loc[:, 'Latitude':'Longitude'].reset_index(drop=True)
df_a
df_a.drop(df_a.index[2])
singapore_merged = pd.concat([df_a, neighborhoods_venues_sorted], axis=1, join='inner')
singapore_merged

# add clustering labels
singapore_merged['Cluster Labels'] = kmeans.labels_

#singapore_merged['Latitude', 'Longitude'] = kmeans.labels_
singapore_merged


,Latitude,Longitude,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,...,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,Cluster Labels
0,1.280386,103.840644,CBD,Café,Korean Restaurant,Hotel,Italian Restaurant,Chinese Restaurant,Food Court,Japanese Restaurant,...,Asian Restaurant,Beer Bar,Beer Garden,Bookstore,Burger Joint,Australian Restaurant,Dessert Shop,Gym / Fitness Center,American Restaurant,0
1,1.278898,103.841915,Marina Area,Korean Restaurant,Café,Italian Restaurant,Chinese Restaurant,Food Court,Coffee Shop,Japanese Restaurant,...,Beer Garden,Beer Bar,Bakery,Australian Restaurant,French Restaurant,Burger Joint,American Restaurant,Vegetarian / Vegan Restaurant,Tea Room,2
2,1.281756,103.841699,Orchard,Chinese Restaurant,Korean Restaurant,Hostel,Cocktail Bar,Café,Spa,Japanese Restaurant,...,Coffee Shop,Hotel,Wine Bar,Vegetarian / Vegan Restaurant,Asian Restaurant,French Restaurant,Tea Room,Beer Bar,Dessert Shop,1


In [105]:
address = 'Singapore, Singapore'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 1.3408528, 103.878446863736.


Finally, let's visualize the resulting clusters

In [111]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(singapore_merged['Latitude'], singapore_merged['Longitude'], singapore_merged['Neighborhood'], singapore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##  Examine Clusters

#### Cluster 1

In [114]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 0, singapore_merged.columns[[2] + list(range(3, singapore_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,Cluster Labels
0,CBD,Café,Korean Restaurant,Hotel,Italian Restaurant,Chinese Restaurant,Food Court,Japanese Restaurant,Bakery,Cocktail Bar,...,Asian Restaurant,Beer Bar,Beer Garden,Bookstore,Burger Joint,Australian Restaurant,Dessert Shop,Gym / Fitness Center,American Restaurant,0


In [115]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 1, singapore_merged.columns[[2] + list(range(3, singapore_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,Cluster Labels
2,Orchard,Chinese Restaurant,Korean Restaurant,Hostel,Cocktail Bar,Café,Spa,Japanese Restaurant,Food Court,Seafood Restaurant,...,Coffee Shop,Hotel,Wine Bar,Vegetarian / Vegan Restaurant,Asian Restaurant,French Restaurant,Tea Room,Beer Bar,Dessert Shop,1


In [116]:
singapore_merged.loc[singapore_merged['Cluster Labels'] == 2, singapore_merged.columns[[2] + list(range(3, singapore_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,Cluster Labels
1,Marina Area,Korean Restaurant,Café,Italian Restaurant,Chinese Restaurant,Food Court,Coffee Shop,Japanese Restaurant,Cocktail Bar,Dessert Shop,...,Beer Garden,Beer Bar,Bakery,Australian Restaurant,French Restaurant,Burger Joint,American Restaurant,Vegetarian / Vegan Restaurant,Tea Room,2
